In [1]:
import os, sys
import json
import tweepy
import tarfile
import shutil

In [2]:
# ref: http://tt-house.com/2011/02/python-dirs-tar.html

def tar_compress_dir( path ):
    zip_targets = []
    base = os.path.basename(path)
    zipfilepath = os.path.abspath('%s.zip' % path)
    for dirpath, dirnames, filenames in os.walk(path):
        for filename in filenames:
            filepath = os.path.join(dirpath, filename)
            if filepath == zipfilepath:
                continue
            arc_name = os.path.relpath(filepath, os.path.dirname(path))
            zip_targets.append((filepath, arc_name))
        for dirname in dirnames:
            filepath = os.path.join(dirpath, dirname)
            arc_name = os.path.relpath(filepath, os.path.dirname(path)) + os.path.sep
            zip_targets.append((filepath, arc_name))

    _tar = tarfile.open('%s.tar.gz' % path,'w:gz')
    for filepath, name in zip_targets:
        _tar.add(filepath, arcname = name)
    _tar.close()
    shutil.rmtree(path)
    return

class StdOutListener(tweepy.StreamListener):
    def on_data(self, data):
        global stream_num
        global dir_idx
        
        tweet = json.loads(data)       
        # retweet fileter
        if not tweet['retweeted'] and 'RT @' not in tweet['text']:
            if stream_num % 1000 == 0: print( stream_num, tweet["created_at"], tweet["text"] )
            if dir_idx != 0 and ( stream_num % 10000 ) == 0: 
                tar_compress_dir( "%s/%d" % ( query_dir, dir_idx ) ) # 圧縮
            if ( stream_num % 10000 ) == 0:
                dir_idx += 1
                new_dir = "%s/%d" % ( query_dir, dir_idx )
                os.mkdir(new_dir)

            save_path = "%s/%d/%s.json" % ( query_dir, dir_idx, tweet["id_str"] )
            f = open(save_path, "w")
            json.dump(tweet, f)
            f.close()
            stream_num += 1
            return True
        return True

    def on_error(self, status):
        print( status )

In [3]:
query = "#欅坂46"

In [4]:
with open( '../data/twitterapi.json', 'r') as f:
    key_dic = json.load(f)
key_id = "1"
savepath =  "../data/tweet"
query_dir = "%s/%s" % (savepath, query)
if not os.path.exists( query_dir ):
    os.mkdir(query_dir)

In [ ]:
stream_num = 0
dir_idx = 0

l = StdOutListener()
auth = tweepy.OAuthHandler( key_dic[key_id]["app_key"], key_dic[key_id]["app_secret"] )
auth.set_access_token( key_dic[key_id]["oauth_token"], key_dic[key_id]["oauth_token_secret"] )

def start_stream():
    while True :
        try:
            stream = tweepy.Stream(auth, l)
            stream.filter(track = [query])
        except:
            continue

print( 'target query:', query )
start_stream()

target query: #欅坂46
